### Fetch category pages for depth=1

In [1]:
import requests
import json
import os

pageTitleSet = set()
pageIdSet = set()
categoryIdSet = set()

def fetch_category_pages(category, filepath, depth):
    if depth > 1:
        return
    next_depth = depth + 1
    
    # Create the directory for the category if it doesn't exist
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    # Make the initial request to get the category information
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:{category}&cmlimit=max"
    headers = {"User-Agent": "Tinker/0.1 (kartikeyapophali@gmail.com)"}
    response = requests.get(url, headers=headers)
    data = response.json()
    
    # Write the response to a response.json file
    response_filepath = os.path.join(filepath, "response.json")
    with open(response_filepath, "w") as file:
        json.dump(data, file)

    # Check if the category has any pages
    if "categorymembers" in data["query"]:
        pages = data["query"]["categorymembers"]

        # Filter out pages and subcategories with 'bird' or 'Bird' in the title
        pages = [page for page in pages if 'bird' not in page["title"].lower()]

        # Separate pages and subcategories
        actual_pages = [page for page in pages if page["ns"] == 0]
        subcategories = [page for page in pages if page["ns"] == 14]

        # Process actual pages
        for page in actual_pages:
            page_title = page["title"]
            pageTitleSet.add(page_title)
            page_id = page["pageid"]
            pageIdSet.add(page_id)

        # Process subcategories
        for subcategory in subcategories:
            subcategory_id = subcategory["pageid"]
            if subcategory_id in categoryIdSet:
                continue
            categoryIdSet.add(subcategory_id)
            subcategory_name = subcategory["title"].replace("Category:", "")
            subcategory_path = os.path.join(filepath, subcategory_name)
            # Recursively fetch pages from subcategories
            fetch_category_pages(subcategory_name, subcategory_path, next_depth)

In [2]:
# fetch_category_pages("Dinosaurs", "/aux/data/wiki/Dinosaurs", -1000)
fetch_category_pages("Dinosaurs", "Dinosaurs", 0)
print(f"Count of pages by page titles: {len(pageTitleSet)}")
print(f"Count of pages by page id: {len(pageIdSet)}")
print(f"Count of categories: {len(categoryIdSet)}")

Count of pages by page titles: 127
Count of pages by page id: 127
Count of categories: 46
